<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/text2speech/vits_finetuning_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/SayaSS/vits-finetuning.git
%cd vits-finetuning

In [ ]:
!pip install -r requirements.txt

In [4]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

/content/vits-finetuning/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/vits-finetuning/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.8/dist-packages/numpy/core/include -I/usr/include/python3.8 -c core.c -o build/temp.linux-x86_64-3.8/core.o
creating /content/vits-finetuning/monotonic_align/monotonic_align
x86_64-linux-gnu-gc

In [5]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#@title Download pre-trained model
!wget -nc -P ../drive/MyDrive/vits-finetune/checkpoints https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0.pth
!wget -nc -P ../drive/MyDrive/vits-finetune/checkpoints https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0.pth

--2023-02-27 02:35:20--  https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0.pth
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/73/57/7357194223863ef0c28d9dd59386c0c4933f0105f0b51e2d107958e679c17cd6/139231259de2084dff7e6b1f8cda5cc488bad74fa0c493b7317d7f43b614f5d8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_0.pth%3B+filename%3D%22G_0.pth%22%3B&Expires=1677724521&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzczLzU3LzczNTcxOTQyMjM4NjNlZjBjMjhkOWRkNTkzODZjMGM0OTMzZjAxMDVmMGI1MWUyZDEwNzk1OGU2NzljMTdjZDYvMTM5MjMxMjU5ZGUyMDg0ZGZmN2U2YjFmOGNkYTVjYzQ4OGJhZDc0ZmEwYzQ5M2I3MzE3ZDdmNDNiNjE0ZjVkOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb2

In [7]:
!python preprocess.py --filelists filelists/miyu_train.txt filelists/miyu_val.txt

START: filelists/miyu_train.txt
Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"
dic.tar.gz: 100% 22.6M/22.6M [00:04<00:00, 5.16MB/s]
Extracting tar file /usr/local/lib/python3.8/dist-packages/pyopenjtalk/dic.tar.gz
START: filelists/miyu_val.txt


In [8]:
#@title  Edit config
import json
batchsize = 16  #@param {type:"number"}
epochs = 1000 #@param{type:"number"}
training_files = "filelists/miyu_train.txt.cleaned" #@param {type:"string"}
validation_files = "filelists/miyu_val.txt.cleaned" #@param {type:"string"}
config = json.load(open("configs/config.json"))
config['train']['batch_size'] = batchsize
config['train']['epochs'] = epochs
config['data']['training_files'] = training_files
config['data']['validation_files'] = validation_files
with open("configs/config.json", 'w+') as f:
    json.dump(config, f, indent=4)

In [ ]:
#@title  Start training
# epoch 1000
!python train_ms.py -c configs/config.json -m checkpoints

INFO:checkpoints:{'train': {'log_interval': 200, 'eval_interval': 1000, 'seed': 1234, 'epochs': 2000, 'learning_rate': 0.0002, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 16, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 8192, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'training_files': 'filelists/miyu_train.txt.cleaned', 'validation_files': 'filelists/miyu_val.txt.cleaned', 'text_cleaners': ['japanese_cleaners'], 'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 804, 'cleaned_text': True}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2], 'upsample

In [ ]:
#@title  Infer
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch


import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
config_path = "configs/config.json" #@param {type:"string"}
model_path = "../drive/MyDrive/vits-finetune/checkpoints/G_1000.pth" #@param {type:"string"}
hps = utils.get_hparams_from_file(config_path)
net_g = SynthesizerTrn(
    len(hps.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
model = net_g.eval()
model = utils.load_checkpoint(model_path, net_g, None)

In [ ]:
speaker_id = 10 #@param {type:"number"}
text = "\u4ECA\u65E5\u306F\u6771\u4EAC\u99C5\u306B\u3044\u307E\u3059\u3002\u3069\u3053\u306B\u884C\u304F\u306E\u304C\u304A\u3059\u3059\u3081\u3067\u3059\u304B\uFF1F" #@param {type:"string"}
noise_scale=0.6 #@param {type:"number"}
noise_scale_w=0.668 #@param {type:"number"}
length_scale=1.0 #@param {type:"number"}
stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([speaker_id]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))